# Federated Learning Example - Binary Classifier with Regression - Automobile Dataset

[Link to dataset source](https://archive.ics.uci.edu/dataset/10/automobile)

[Link to Colab (deprecated)](https://colab.research.google.com/drive/1GmAhxnKVvrhWffospDEe0rc-QB_tjfhE?usp=sharing)


In [1]:
import os
import random
import numpy as np
from pathlib import Path
from tensorflow import convert_to_tensor
import tensorflow as tf
import matplotlib.pyplot as plt
from src.tf_utils import df_to_tfds
from src.data_examples.ex1_data_loader import ExampleDataLoader
from src.data_examples.ex1_build import build_and_evaluate


In [2]:
# an attempt to suppress warnings flooding notebook stdout

import logging
import warnings
import absl.logging

tf.get_logger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

absl.logging.set_verbosity(absl.logging.ERROR)


In [3]:
# globals const

RAND_SEED   = 1337
N_EPOCH     = 25
BATCH_SIZE  = 24

In [4]:
random.seed(RAND_SEED)

saved_model_path = Path('saved_models')
saved_model_path.mkdir(exist_ok=True)
metrics_csv_path = Path('metrics')
metrics_csv_path.mkdir(exist_ok=True)

In [5]:
all_result_histories = {}
all_result_models = {}
all_model_names = [] # for bookkeeping

In [6]:
data = ExampleDataLoader()
data.download().load().clean()

data.df['symboling_threshold'] = [1 if i > 0 else 0 for i in data.df['symboling']]

target_feature_label = 'symboling_threshold'

using cached file cache\static\public\10\automobile.zip
extracting zip file content:
 	size: 144	filename: Index
 	size: 1197	filename: app.css
 	size: 25936	filename: imports-85.data
 	size: 4747	filename: imports-85.names
 	size: 3757	filename: misc


In [7]:
def generate_random_sample_from_spec(data_spec, features_override=[]):
  ret = {}
  for k in data_spec.keys():
    if features_override and k not in features_override:
      continue
    v = data_spec.get(k)
    if isinstance(v, tuple):
      ret[k] = random.random() * (v[1] - v[0])
    elif isinstance(v, list):
      ret[k] = random.choice(v)
    else:
      ret[k] = v
  return ret

In [8]:
__inference_sample_spec = list(map(lambda x: x.replace('_', '-'), data.features_categorical + data.features_numeric_continuous))
__inference_sample = generate_random_sample_from_spec(data.data_spec, __inference_sample_spec)
inference_sample = {}
for k, v in __inference_sample.items():
  inference_sample[k.replace('-', '_')] = convert_to_tensor([v])

inference_sample

{'make': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'toyota'], dtype=object)>,
 'fuel_type': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'gas'], dtype=object)>,
 'aspiration': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'std'], dtype=object)>,
 'num_of_doors': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'two'], dtype=object)>,
 'body_style': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'hatchback'], dtype=object)>,
 'drive_wheels': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'rwd'], dtype=object)>,
 'engine_location': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'rear'], dtype=object)>,
 'wheel_base': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([31.61522], dtype=float32)>,
 'length': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([20.611437], dtype=float32)>,
 'width': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([11.906965], dtype=float32)>,
 'height': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.459352],

### Splitting dataframe into train-validation and test sets

Splitting `train-validation` with `test` is performed so model testing accuracy performance is done using same/shared global dataset

In [9]:
df_main_train_val = data.df.sample(frac=0.9, random_state=RAND_SEED)
df_main_test      = data.df.drop(df_main_train_val.index)

tfds_main_test    = df_to_tfds(df_main_test,  target_feature_label, batch_size=8, shuffle=False)


In [10]:
df_main_test

,symboling,normalized_losses,make,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,...,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price,symboling_threshold
12,0,188.0,bmw,gas,std,two,sedan,rwd,front,101.199997,...,mpfi,3.31,3.19,9.0,121.0,4250.0,21.0,28.0,20970.0,0
19,1,98.0,chevrolet,gas,std,two,hatchback,fwd,front,94.500000,...,2bbl,3.03,3.11,9.6,70.0,5400.0,38.0,43.0,6295.0,1
33,1,101.0,honda,gas,std,two,hatchback,fwd,front,93.699997,...,1bbl,2.91,3.41,9.2,76.0,6000.0,30.0,34.0,6529.0,1
34,1,101.0,honda,gas,std,two,hatchback,fwd,front,93.699997,...,1bbl,2.91,3.41,9.2,76.0,6000.0,30.0,34.0,7129.0,1
38,0,106.0,honda,gas,std,two,hatchback,fwd,front,96.500000,...,1bbl,3.15,3.58,9.0,86.0,5800.0,27.0,33.0,9095.0,0
102,0,108.0,nissan,gas,std,four,wagon,fwd,front,100.400002,...,mpfi,3.43,3.27,9.0,152.0,5200.0,17.0,22.0,14399.0,0
116,0,161.0,peugot,diesel,turbo,four,sedan,rwd,front,107.900002,...,idi,3.70,3.52,21.0,95.0,4150.0,28.0,33.0,17950.0,0
118,1,119.0,plymouth,gas,std,two,hatchback,fwd,front,93.699997,...,2bbl,2.97,3.23,9.4,68.0,5500.0,37.0,41.0,5572.0,1
125,3,186.0,porsche,gas,std,two,hatchback,rwd,front,94.500000,...,mpfi,3.94,3.11,9.5,143.0,5500.0,19.0,27.0,22018.0,1
133,2,104.0,saab,gas,std,four,sedan,fwd,front,99.099998,...,mpfi,3.54,3.07,9.3,110.0,5250.0,21.0,28.0,12170.0,1


## 1. Centralized (Conventional) Training 

In [11]:

model_name = 'ex1ch1_auto_classifier_centralized'

ex1ch1_model_path = saved_model_path / model_name

all_result_histories[model_name] = []
all_result_models[model_name] = []
all_model_names.append(model_name)

In [12]:
df_train    = df_main_train_val.sample(frac=0.8, random_state=RAND_SEED)
df_val      = df_main_train_val.drop(df_train.index)

tfds_train  = df_to_tfds(df_train, target_feature_label, batch_size=BATCH_SIZE)
tfds_val    = df_to_tfds(df_val,   target_feature_label, batch_size=BATCH_SIZE)

df_train.shape, df_val.shape

((114, 27), (29, 27))

In [13]:
res_model, logger, history = build_and_evaluate(
    tfds_train,
    tfds_val,
    data,
    epoch=N_EPOCH,
    model_name=model_name,
    csv_metrics_filepath=metrics_csv_path
  )

logging to metrics\ex1ch1_auto_classifier_centralized_training_metrics.csv


In [14]:
res_model.save(ex1ch1_model_path)
all_result_histories[model_name].append(history)
all_result_models[model_name].append(res_model)

loss, accuracy, mse, ba = res_model.evaluate(tfds_main_test)
print(
  f'Loss:\t{loss}',
  f'Accuracy:\t{accuracy}',
  f'MSE:\t{mse}', 
  f'Binary Accuracy:\t{ba}',
  sep='\n' 
)

2/2 [==============================] - 0s 4ms/step - loss: 1.2475 - accuracy: 0.7500 - mse: 0.2029 - binary_accuracy: 0.7500
Loss:	1.2475287914276123
Accuracy:	0.75
MSE:	0.20285019278526306
Binary Accuracy:	0.75


## 2. Federated Model - Model Ensembling

Scope:
- Client(s) will be treated as such they have similar hardware, model and training strategy
- Client(s) and the resulting model is stored as iterable list, accessed by index
- Training is not performed asynchronously
- Distribution of dataset is not based on other parameters, and is considered as random

Evaluation helper function for ensemble model as follows

In [15]:
def fn_eval(model, tfds):
  # valuation for metrics
  return model.evaluate(tfds)

def fn_predict(model, tfds):
  # valuation for prediction
  return model.predict(tfds)

def evaluate_ensemble(models, tfds, fn_valuation, fn_aggregation=np.mean):
  res = []
  for model in models:
    res.append(fn_valuation(model, tfds))
  return fn_aggregation(res, axis=0)


### 2.1 Evenly Distributed Dataset

In [16]:
n_client = 5

model_name = 'ex1ch1_auto_classifier_federated_ensemble_n5'

ex1ch2_model_path = saved_model_path / model_name

all_result_histories[model_name] = []
all_result_models[model_name] = []
all_model_names.append(model_name)


In [17]:
__scoped_metrics = []

for n, data_df in enumerate(np.array_split(data.df, n_client)):

  client_model_name = f'{model_name}_{n}'
  ex1ch2_model_path_c = saved_model_path / client_model_name 

  _df_train    = df_main_train_val.sample(frac=0.8, random_state=RAND_SEED)
  _df_val      = df_main_train_val.drop(df_train.index)

  _tfds_train  = df_to_tfds(_df_train, target_feature_label, batch_size=BATCH_SIZE)
  _tfds_val    = df_to_tfds(_df_val,   target_feature_label, batch_size=BATCH_SIZE)

  res_model, logger, history = build_and_evaluate(
    _tfds_train,
    _tfds_val,
    data,
    epoch=N_EPOCH,
    model_name=client_model_name,
    csv_metrics_filepath=metrics_csv_path
  )

  res_model.save(ex1ch2_model_path_c)
  all_result_histories[model_name].append(history)
  all_result_models[model_name].append(res_model)

  __scoped_metrics.append(res_model.evaluate(tfds_main_test))


for n, (loss, accuracy, mse, ba)  in enumerate(__scoped_metrics):
  print('---------------------------------')
  print(f'{model_name} {n}')
  print(
    f'Loss:\t{loss}',
    f'Accuracy:\t{accuracy}',
    f'MSE:\t{mse}', 
    f'Binary Accuracy:\t{ba}',
    sep='\n' 
  )

logging to metrics\ex1ch1_auto_classifier_federated_ensemble_0_training_metrics.csv
2/2 [==============================] - 0s 4ms/step - loss: 0.4181 - accuracy: 0.6875 - mse: 0.1577 - binary_accuracy: 0.6875
logging to metrics\ex1ch1_auto_classifier_federated_ensemble_1_training_metrics.csv
2/2 [==============================] - 0s 5ms/step - loss: 0.5215 - accuracy: 0.7500 - mse: 0.1732 - binary_accuracy: 0.7500
logging to metrics\ex1ch1_auto_classifier_federated_ensemble_2_training_metrics.csv
2/2 [==============================] - 0s 5ms/step - loss: 0.4075 - accuracy: 0.8750 - mse: 0.1383 - binary_accuracy: 0.8750
logging to metrics\ex1ch1_auto_classifier_federated_ensemble_3_training_metrics.csv
2/2 [==============================] - 0s 5ms/step - loss: 3.0698 - accuracy: 0.6875 - mse: 0.5342 - binary_accuracy: 0.6875
logging to metrics\ex1ch1_auto_classifier_federated_ensemble_4_training_metrics.csv
2/2 [==============================] - 0s 5ms/step - loss: 0.5588 - accuracy: 0.

In [18]:
print('ensemble result for', model_name)
loss, accuracy, mse, ba = evaluate_ensemble(all_result_models[model_name], tfds_main_test, fn_eval)

print(
  f'Loss:\t{loss}',
  f'Accuracy:\t{accuracy}',
  f'MSE:\t{mse}', 
  f'Binary Accuracy:\t{ba}',
  sep='\n' 
)

evaluate_ensemble(all_result_models[model_name], tfds_main_test, fn_predict)

ensemble result for ex1ch1_auto_classifier_federated_ensemble
2/2 [==============================] - 0s 6ms/step - loss: 0.5215 - accuracy: 0.7500 - mse: 0.1732 - binary_accuracy: 0.7500


2/2 [==============================] - 0s 5ms/step - loss: 0.5588 - accuracy: 0.8125 - mse: 0.1854 - binary_accuracy: 0.8125
Loss:	0.9951283097267151
Accuracy:	0.7625
MSE:	0.23776735067367555
Binary Accuracy:	0.7625
2/2 [==============================] - 0s 3ms/step


array([[ 2.9706970e-01],
       [ 1.0248438e+00],
       [ 1.0342724e+00],
       [ 1.0349839e+00],
       [ 8.2339847e-01],
       [-9.0524851e-04],
       [-1.9033358e-01],
       [ 1.1377199e+00],
       [ 5.8184004e-01],
       [ 2.9238468e-01],
       [ 3.4043735e-01],
       [ 2.9084820e-01],
       [ 5.2077669e-01],
       [ 3.8632697e-01],
       [-5.0788127e-02],
       [-2.4032113e-01]], dtype=float32)

In [22]:
# redo for other number of client

n_clients = [10, 15]

for n_client in n_clients:
  model_name = f'ex1ch1_auto_classifier_federated_ensemble_n{n_client}'

  ex1ch2_model_path = saved_model_path / model_name

  all_result_histories[model_name] = []
  all_result_models[model_name] = []
  all_model_names.append(model_name)

  __scoped_metrics = []

  for n, data_df in enumerate(np.array_split(data.df, n_client)):

    client_model_name = f'{model_name}_{n}'
    ex1ch2_model_path_c = saved_model_path / client_model_name 

    _df_train    = df_main_train_val.sample(frac=0.8, random_state=RAND_SEED)
    _df_val      = df_main_train_val.drop(df_train.index)

    _tfds_train  = df_to_tfds(_df_train, target_feature_label, batch_size=BATCH_SIZE)
    _tfds_val    = df_to_tfds(_df_val,   target_feature_label, batch_size=BATCH_SIZE)

    res_model, logger, history = build_and_evaluate(
      _tfds_train,
      _tfds_val,
      data,
      epoch=N_EPOCH,
      model_name=client_model_name,
      csv_metrics_filepath=metrics_csv_path
    )

    res_model.save(ex1ch2_model_path_c)
    all_result_histories[model_name].append(history)
    all_result_models[model_name].append(res_model)

    __scoped_metrics.append(res_model.evaluate(tfds_main_test))


  for n, (loss, accuracy, mse, ba)  in enumerate(__scoped_metrics):
    print('---------------------------------')
    print(f'{model_name} {n}')
    print(
      f'Loss:\t{loss}',
      f'Accuracy:\t{accuracy}',
      f'MSE:\t{mse}', 
      f'Binary Accuracy:\t{ba}',
      sep='\n' 
    )


  print('ensemble result for', model_name)
  loss, accuracy, mse, ba = evaluate_ensemble(all_result_models[model_name], tfds_main_test, fn_eval)

  print(
    f'Loss:\t{loss}',
    f'Accuracy:\t{accuracy}',
    f'MSE:\t{mse}', 
    f'Binary Accuracy:\t{ba}',
    sep='\n' 
  )


logging to metrics\ex1ch1_auto_classifier_federated_ensemble_n10_0_training_metrics.csv
2/2 [==============================] - 0s 5ms/step - loss: 1.6444 - accuracy: 0.6250 - mse: 0.3442 - binary_accuracy: 0.6250
logging to metrics\ex1ch1_auto_classifier_federated_ensemble_n10_1_training_metrics.csv
2/2 [==============================] - 0s 4ms/step - loss: 6.9102 - accuracy: 0.5000 - mse: 0.7038 - binary_accuracy: 0.5000
logging to metrics\ex1ch1_auto_classifier_federated_ensemble_n10_2_training_metrics.csv
2/2 [==============================] - 0s 5ms/step - loss: 0.4637 - accuracy: 0.7500 - mse: 0.1714 - binary_accuracy: 0.7500
logging to metrics\ex1ch1_auto_classifier_federated_ensemble_n10_3_training_metrics.csv
2/2 [==============================] - 0s 4ms/step - loss: 0.4580 - accuracy: 0.7500 - mse: 0.1670 - binary_accuracy: 0.7500
logging to metrics\ex1ch1_auto_classifier_federated_ensemble_n10_4_training_metrics.csv
2/2 [==============================] - 0s 5ms/step - loss: 1

In [19]:
# hist = result_histories['ex1ch1_auto_classifier_federated_naive'][1]

# plt.plot(hist.history['accuracy'])
# plt.plot(hist.history['val_accuracy'])
# plt.title(f'Model Accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')

# plt.show()

# Validation and Metrics

In [23]:
all_model_names

['ex1ch1_auto_classifier_centralized',
 'ex1ch1_auto_classifier_federated_ensemble',
 'ex1ch1_auto_classifier_federated_ensemble_n{n_client}',
 'ex1ch1_auto_classifier_federated_ensemble_n10',
 'ex1ch1_auto_classifier_federated_ensemble_n15',
 'ex1ch1_auto_classifier_federated_ensemble_n20']